In [2]:
# Install required packages
!pip install --upgrade langchain-google-genai google-generativeai
!pip install fastapi uvicorn langgraph langchain streamlit google-api-python-client google-auth-httplib2 google-auth-oauthlib python-dotenv

# Import necessary libraries
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional
import google.auth
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import os
from langchain.agents import AgentExecutor, Tool
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain_core.messages import SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import google.generativeai as genai

# Configure Google API credentials
def configure_google_apis():
    # For Google Calendar API
    SERVICE_ACCOUNT_FILE = 'service_account.json'  # Path to your service account JSON
    SCOPES = ['https://www.googleapis.com/auth/calendar']

    # For Gemini API
    GOOGLE_API_KEY = 'YOUR_GOOGLE_API_KEY'  # Replace with your actual API key

    # Configure Gemini
    genai.configure(api_key=GOOGLE_API_KEY)

    return SERVICE_ACCOUNT_FILE, SCOPES

SERVICE_ACCOUNT_FILE, SCOPES = configure_google_apis()

# Initialize Google Gemini LLM with explicit API key
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.7,
    google_api_key=os.getenv("GOOGLE_API_KEY") or "YOUR_GOOGLE_API_KEY"  # Replace or use environment variable
)

# Initialize FastAPI (for notebook, we'll mock the API calls)
app = FastAPI()

class AppointmentRequest(BaseModel):
    user_query: str
    time_min: Optional[str] = None
    time_max: Optional[str] = None

def book_appointment(request: AppointmentRequest):
    """Mock function for notebook demo"""
    try:
        creds = google.oauth2.service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        service = build('calendar', 'v3', credentials=creds)

        events_result = service.events().list(
            calendarId='primary',
            timeMin=request.time_min or datetime.utcnow().isoformat() + 'Z',
            timeMax=request.time_max or (datetime.utcnow() + timedelta(days=7)).isoformat() + 'Z',
            singleEvents=True,
            orderBy='startTime'
        ).execute()

        return {"available_slots": events_result.get('items', [])}
    except Exception as e:
        return {"error": str(e)}

# For notebook demo, we'll use mock data
def check_availability(query: str):
    """Mock availability checker for notebook demo"""
    # Parse the query to extract date/time information
    if "tomorrow" in query.lower():
        date = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
        return f"Available slots for {date}: 9 AM, 11 AM, 2 PM, 4 PM"
    elif "tuesday" in query.lower():
        return "Available slots for next Tuesday: 10 AM, 2 PM, 3:30 PM"
    else:
        return "Available slots today: 10 AM, 2 PM, 4 PM"

# Create tools for the agent
tools = [
    Tool(
        name="CheckAvailability",
        func=check_availability,
        description="Checks Google Calendar for available slots. Input should be the desired date/time."
    ),
    Tool(
        name="BookAppointment",
        func=lambda x: "Appointment booked successfully!",
        description="Books an appointment on Google Calendar. Input should be the exact time slot to book."
    )
]

# Initialize the agent
agent = OpenAIFunctionsAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
    system_message=SystemMessage(content="You are an AI assistant that helps users book appointments. Be friendly and helpful.")
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Create a simple chat interface for Jupyter Notebook
print("📅 Calendar Booking Assistant")
print("Type your message below and press Enter to chat with the assistant.")
print("Type 'exit' to end the conversation.\n")

# Initialize conversation memory
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory
)

# Create widgets for the chat interface
chat_output = widgets.Output()
user_input = widgets.Text(
    placeholder='Type your message here...',
    description='You:',
    disabled=False
)
send_button = widgets.Button(description="Send")

def on_button_clicked(b):
    with chat_output:
        user_message = user_input.value
        if user_message.lower() == 'exit':
            print("Conversation ended. Goodbye!")
            return

        print(f"You: {user_message}")

        # Get agent response
        try:
            response = agent_executor.run(user_message)
            print(f"Assistant: {response}")
        except Exception as e:
            print(f"Assistant: I encountered an error. Please try again. ({str(e)})")

        print("-" * 50)

        # Clear input for next message
        user_input.value = ''

send_button.on_click(on_button_clicked)

# Display the widgets
display(widgets.VBox([
    chat_output,
    widgets.HBox([user_input, send_button])
]))

# Sample conversation
print("\nTry these example prompts:")
print("1. I need to book a meeting next Tuesday")
print("2. What times are available tomorrow?")
print("3. Book me for the 2 PM slot")
print("4. Can we make it a 1-hour meeting instead?")

  Using cached langchain_google_genai-2.1.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to pro


Try these example prompts:
1. I need to book a meeting next Tuesday
2. What times are available tomorrow?
3. Book me for the 2 PM slot
4. Can we make it a 1-hour meeting instead?
